In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

from sacred import Experiment

ex = Experiment()

In [ ]:
@ex.config
def cfg():
    batch_size = 128
    num_units_first_layer = 512
    num_units_second_layer = 512
    dropout_first_layer = 0.2
    dropout_second_layer = 0.2
    learning_rate = 0.001


In [ ]:
@ex.automain
def run(batch_size,
        num_units_first_layer,
        num_units_second_layer,
        dropout_first_layer,
        dropout_second_layer,
        learning_rate):

    num_classes = 10
    epochs = 20

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = Sequential()
    model.add(Dense(num_units_first_layer, activation='relu', input_shape=(784,)))
    model.add(Dropout(dropout_first_layer))
    model.add(Dense(num_units_second_layer, activation='relu'))
    model.add(Dropout(dropout_second_layer))
    model.add(Dense(10, activation='softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(lr=learning_rate),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    results = dict()
    results["optimization_target"] = 1 - score[1]

    return results